In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt 
import nltk
nltk.download('vader_lexicon')


In [17]:
finviz_url = 'https://finviz.com/quote.ashx?t='
tickers = ['MSFT', 'GOOG', 'TSLA']

news_tables = {}
for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url=url, headers={'user-agent': 'my-app'})
    response = urlopen(req)

    html = BeautifulSoup(response, features='html.parser')
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table




In [18]:
parsed_data = []

for ticker, news_table in news_tables.items():

    for row in news_table.findAll('tr'):

        title = row.a.text
        date_data = row.td.text.split(' ')

        if len(date_data) == 1:
            time = date_data[0]
        else:
            date = date_data[0]
            time = date_data[1]

        parsed_data.append([ticker, date, time, title])

df = pd.DataFrame(parsed_data, columns=['ticker', 'date', 'time', 'title'])


/tmp/ipykernel_6480/2523268079.py:5: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  for row in news_table.findAll('tr'):


In [1]:
# Now you can initialize VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()#
# 
#  Initialize VADER
vader = SentimentIntensityAnalyzer()

# Calculate compound sentiment scores
df['compound'] = df['title'].apply(lambda x: vader.polarity_scores(str(x))['compound'])

# Convert dates to datetime
df['date'] = pd.to_datetime(df['date']).dt.date

# Create the visualization
plt.figure(figsize=(12, 6))

# Group by ticker and date, calculate mean sentiment
mean_df = df.groupby(['ticker', 'date'])['compound'].mean().unstack()

# Plot the results
ax = mean_df.plot(kind='bar', width=0.8)

# Customize the plot
plt.title('Average Sentiment Score by Ticker Over Time', pad=20)
plt.xlabel('Ticker')
plt.ylabel('Average Sentiment Score')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Add grid for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Show the plot
plt.show()

NameError: name 'df' is not defined